In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#  理解数据

#  理解数据

### 用户信息表

In [45]:
user_info = pd.read_csv(r"data/data_format1/user_info_format1.csv")
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [46]:
user_info.user_id = user_info.user_id.astype("str")
user_info.age_range = user_info.age_range.astype("str")
user_info.gender = user_info.gender.astype("str")
user_info.describe()

,user_id,age_range,gender
count,424170,424170,424170
unique,424170,10,4
top,376517,3.0,0.0
freq,1,111654,285638


In [47]:
user_info.user_id = user_info.user_id.astype("int64")
user_info.age_range = user_info.age_range.astype("float")
user_info.gender = user_info.gender.astype("float")
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    424170 non-null  int64  
 1   age_range  421953 non-null  float64
 2   gender     417734 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 9.7 MB


In [48]:
#删除缺失值
user_info.dropna(axis=0,inplace=True)

In [49]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417708 entries, 0 to 424169
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    417708 non-null  int64  
 1   age_range  417708 non-null  float64
 2   gender     417708 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 12.7 MB


In [50]:
user_info.duplicated().sum()
#user_info.drop_duplicates(inplace=True)

0

In [51]:
user_info.gender.value_counts()

0.0    285634
1.0    121655
2.0     10419
Name: gender, dtype: int64

In [52]:
user_info.age_range.value_counts()

3.0    110952
0.0     90638
4.0     79649
2.0     52420
5.0     40601
6.0     35257
7.0      6924
8.0      1243
1.0        24
Name: age_range, dtype: int64

In [53]:
#删除缺失值 gender字段的2为缺失值，　age_range 的0也为缺失值 
user_info = user_info.loc[(user_info.gender != 2) & (user_info.age_range != 0),:]

In [54]:
user_info.age_range[user_info.age_range == 8] =7

###  用户商铺复购表

In [55]:
user_merchant = pd.read_csv(r"data/data_format1/train_format1.csv")
user_merchant.head()

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [56]:
user_merchant.groupby('merchant_id').size()

merchant_id
2        59
8        22
9        31
10      456
13      181
       ... 
4987     36
4988     24
4991     35
4992    486
4993    131
Length: 1993, dtype: int64

In [57]:
# 查看标签比例
user_merchant.label.value_counts()

0    244912
1     15952
Name: label, dtype: int64

In [58]:
15952/user_merchant.shape[0]

0.06115063788027478

In [59]:
user_merchant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260864 entries, 0 to 260863
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   user_id      260864 non-null  int64
 1   merchant_id  260864 non-null  int64
 2   label        260864 non-null  int64
dtypes: int64(3)
memory usage: 6.0 MB


In [60]:
user_merchant.user_id = user_merchant.user_id.astype("str")
user_merchant.merchant_id  = user_merchant.merchant_id .astype("str")
user_merchant.label  = user_merchant.label .astype("str")
user_merchant.describe(include="all")

,user_id,merchant_id,label
count,260864,260864,260864
unique,212062,1993,2
top,221133,4044,0
freq,18,3379,244912


In [61]:
user_merchant.describe(include="all")

,user_id,merchant_id,label
count,260864,260864,260864
unique,212062,1993,2
top,221133,4044,0
freq,18,3379,244912


In [93]:
#少类小于10%，我们就认为是不平衡数据了
from imblearn.under_sampling  import NearMiss
ee =NearMiss(version=1)
X_resampled, y_resampled = ee.fit_resample(user_merchant.loc[:,["user_id","merchant_id"]], user_merchant.label)
user_merchant =pd.concat([X_resampled,y_resampled],axis=1)
user_merchant.head()

,user_id,merchant_id,label
0,251021,3609,0
1,251038,3624,0
2,314568,3828,0
3,314580,3853,0
4,339322,1399,0


,user_id,merchant_id,label
0,251021,3609,0
1,251038,3624,0
2,314568,3828,0
3,314580,3853,0
4,339322,1399,0


In [86]:
user_merchant.user_id = user_merchant.user_id.astype("int64")
user_merchant.merchant_id  = user_merchant.merchant_id .astype("int64")
user_merchant.label  = user_merchant.label .astype("int64")
user_merchant.info()

NameError: name 'user_merc' is not defined

In [64]:
y_resampled.value_counts()

0    15952
1    15952
Name: label, dtype: int64

In [65]:
# 利用用户商铺表失衡数据处理的结果过滤用户信息表
user_info = user_info.loc[user_info.user_id.isin(user_merchant.user_id.unique()),:]

###  用户行为表

In [66]:
user_log = pd.read_csv(r"data/data_format1/user_log_format1.csv")

In [67]:

user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [68]:
user_log.time_stamp.unique()
# user_log.describe()
user_log.time_stamp

0            829
1            829
2            829
3            829
4            829
            ... 
54925325    1110
54925326    1110
54925327    1110
54925328    1110
54925329    1111
Name: time_stamp, Length: 54925330, dtype: int64

In [69]:
# 利用用户商铺表失衡数据处理的结果过滤用户行为表
user_log = user_log.loc[user_log.user_id.isin(user_merchant.user_id.values),:]
user_log = user_log.loc[user_log.seller_id.isin(user_merchant.merchant_id.values),:]

In [70]:
user_log.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1972402 entries, 1691 to 54924616
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   seller_id    int64  
 4   brand_id     float64
 5   time_stamp   int64  
 6   action_type  int64  
dtypes: float64(1), int64(6)
memory usage: 120.4 MB


In [71]:
user_log.user_id = user_log.user_id.astype("str")
user_log.item_id = user_log.item_id.astype("str")
user_log.cat_id = user_log.cat_id.astype("str")
user_log.seller_id = user_log.seller_id.astype("str")
user_log.brand_id = user_log.brand_id.astype("str")
user_log.time_stamp = user_log.time_stamp.astype("str")
user_log.action_type = user_log.action_type.astype("str")
user_log.describe(include="all")

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
count,1972402,1972402,1972402,1972402,1972402,1972402,1972402
unique,30629,193324,1180,1912,3358,185,4
top,391188,269715,662,3828,1446.0,1111,0
freq,3954,4473,182958,76723,76649,440218,1732606


In [72]:
user_log.user_id = user_log.user_id.astype("int64")
user_log.item_id = user_log.item_id.astype("int64")
user_log.cat_id = user_log.cat_id.astype("int64")
user_log.seller_id = user_log.seller_id.astype("int64")
user_log.brand_id = user_log.brand_id.astype("float")
user_log.time_stamp = user_log.time_stamp.astype("int64")
user_log.action_type = user_log.action_type.astype("int64")

In [73]:
#计算缺失值，及趋缺失占比
user_log.apply(lambda x:[x.isnull().sum(),x.isnull().sum()/x.size], axis=0)  

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,0.0,0.0,0.0,0.0,2938.00000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00149,0.0,0.0


In [74]:
#删除缺失值
user_log = user_log[user_log.brand_id.isna()==False]

In [75]:
user_log.shape

(1969464, 7)

# 数据准备
###  用户特征提取

In [76]:
#用户交互总次数
user_feaut = user_log.groupby("user_id")["action_type"].count().to_frame()
user_feaut.head()

,action_type
user_id,
1,21
4,42
7,6
14,313
17,23


In [77]:
#用户各种行为总次数统计（点击、加购、收藏和购买）
user_feaut_2 =  pd.pivot_table(user_log,index="user_id",columns="action_type",values="cat_id",aggfunc="count")
user_feaut_2.head()

action_type,0,1,2,3
user_id,,,,
1,16.0,NaN,5.0,NaN
4,41.0,NaN,1.0,NaN
7,2.0,NaN,4.0,NaN
14,290.0,NaN,18.0,5.0
17,18.0,NaN,1.0,4.0


In [78]:
user_feaut = user_feaut.merge(user_feaut_2,on="user_id")
user_feaut.head()

,action_type,0,1,2,3
user_id,,,,,
1,21,16.0,NaN,5.0,NaN
4,42,41.0,NaN,1.0,NaN
7,6,2.0,NaN,4.0,NaN
14,313,290.0,NaN,18.0,5.0
17,23,18.0,NaN,1.0,4.0


In [79]:
user_feaut.columns =["total_log","click","add_car","buy","collect"]
user_feaut.head()

,total_log,click,add_car,buy,collect
user_id,,,,,
1,21,16.0,NaN,5.0,NaN
4,42,41.0,NaN,1.0,NaN
7,6,2.0,NaN,4.0,NaN
14,313,290.0,NaN,18.0,5.0
17,23,18.0,NaN,1.0,4.0


In [80]:
#用户交互了多少商铺数多少种商品，多少商品类别和多少商品品牌数量
user_feaut_2 = user_log.groupby("user_id")["seller_id","item_id","cat_id","brand_id"].nunique()
user_feaut_2.head()

/var/folders/tl/smm02s2s3fb6l87fkyb_mmnm0000gn/T/ipykernel_96743/460368127.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  user_feaut_2 = user_log.groupby("user_id")["seller_id","item_id","cat_id","brand_id"].nunique()


,seller_id,item_id,cat_id,brand_id
user_id,,,,
1,4,4,3,4
4,6,20,10,6
7,2,5,3,2
14,40,177,34,38
17,4,8,5,4


In [81]:
user_feaut_2.columns = ["seller_count","item_count","cat_count","brand_count"]
user_feaut_2.head()

,seller_count,item_count,cat_count,brand_count
user_id,,,,
1,4,4,3,4
4,6,20,10,6
7,2,5,3,2
14,40,177,34,38
17,4,8,5,4


In [82]:
user_feaut = user_feaut.merge(user_feaut_2,on="user_id") 
user_feaut.head()

,total_log,click,add_car,buy,collect,seller_count,item_count,cat_count,brand_count
user_id,,,,,,,,,
1,21,16.0,NaN,5.0,NaN,4,4,3,4
4,42,41.0,NaN,1.0,NaN,6,20,10,6
7,6,2.0,NaN,4.0,NaN,2,5,3,2
14,313,290.0,NaN,18.0,5.0,40,177,34,38
17,23,18.0,NaN,1.0,4.0,4,8,5,4


In [79]:
#从时间戳中提月和日
user_log["month"] = user_log.time_stamp // 100;
user_log["day"] = user_log.time_stamp % 100;

In [80]:
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,month,day
1691,26516,416965,1401,586,5579.0,1107,0,11,7
1692,26516,416965,1401,586,5579.0,1107,0,11,7
1693,26516,416965,1401,586,5579.0,1107,0,11,7
1694,26516,352345,177,2565,8149.0,1101,0,11,1
1718,26516,142231,177,2565,8149.0,1101,0,11,1


In [81]:
# 用户平均每天交互、购买的次数 ，用户平均每月交互、购买的次数
user_feaut_2=user_log.groupby("user_id")["month","time_stamp"].nunique()
user_feaut_2.columns = ["month_count","day_count"]
user_feaut_2.head()

,month_count,day_count
user_id,,
1,2,3
4,5,9
7,2,3
14,7,48
17,3,6


In [82]:
user_feaut = user_feaut.merge(user_feaut_2,on="user_id") 
user_feaut.head()

,total_log,click,add_car,buy,collect,seller_count,item_count,cat_count,brand_count,month_count,day_count
user_id,,,,,,,,,,,
1,21,16.0,NaN,5.0,NaN,4,4,3,4,2,3
4,42,41.0,NaN,1.0,NaN,6,20,10,6,5,9
7,6,2.0,NaN,4.0,NaN,2,5,3,2,2,3
14,313,290.0,NaN,18.0,5.0,40,177,34,38,7,48
17,23,18.0,NaN,1.0,4.0,4,8,5,4,3,6


In [83]:
user_feaut["month_avg_log"] = user_feaut.total_log/user_feaut.month_count
user_feaut["month_avg_buy"] =np.where(user_feaut.buy.isna(),0,user_feaut.buy/user_feaut.month_count)
user_feaut["day_avg_log"] = user_feaut.total_log/user_feaut.day_count
user_feaut["day_avg_buy"] =np.where(user_feaut.buy.isna(),0,user_feaut.buy/user_feaut.day_count)
user_feaut.head()

,total_log,click,add_car,buy,collect,seller_count,item_count,cat_count,brand_count,month_count,day_count,month_avg_log,month_avg_buy,day_avg_log,day_avg_buy
user_id,,,,,,,,,,,,,,,
1,21,16.0,NaN,5.0,NaN,4,4,3,4,2,3,10.500000,2.500000,7.000000,1.666667
4,42,41.0,NaN,1.0,NaN,6,20,10,6,5,9,8.400000,0.200000,4.666667,0.111111
7,6,2.0,NaN,4.0,NaN,2,5,3,2,2,3,3.000000,2.000000,2.000000,1.333333
14,313,290.0,NaN,18.0,5.0,40,177,34,38,7,48,44.714286,2.571429,6.520833,0.375000
17,23,18.0,NaN,1.0,4.0,4,8,5,4,3,6,7.666667,0.333333,3.833333,0.166667


In [84]:
#删除特征提取的临时数据框
del user_feaut_2

### 商铺特征提取

In [85]:
#商铺下所有交互总次数
shop_feaut = user_log.groupby("seller_id")["user_id"].count().to_frame()
shop_feaut.head()

,user_id
seller_id,
2,214
8,305
9,242
10,1658
13,681


In [86]:
shop_feaut.columns=["total_count"]
shop_feaut.head()

,total_count
seller_id,
2,214
8,305
9,242
10,1658
13,681


In [87]:
#商铺下各种行为总次数统计（点击、加购、收藏和购买）
shop_feaut_2 = pd.pivot_table(user_log,index="seller_id",columns="action_type",values="user_id",aggfunc="count")
shop_feaut_2.head()

action_type,0,1,2,3
seller_id,,,,
2,181.0,NaN,23.0,10.0
8,258.0,NaN,30.0,17.0
9,219.0,NaN,14.0,9.0
10,1529.0,NaN,72.0,57.0
13,567.0,NaN,100.0,14.0


In [88]:
shop_feaut= shop_feaut.merge(shop_feaut_2,on="seller_id")
shop_feaut.head()

,total_count,0,1,2,3
seller_id,,,,,
2,214,181.0,NaN,23.0,10.0
8,305,258.0,NaN,30.0,17.0
9,242,219.0,NaN,14.0,9.0
10,1658,1529.0,NaN,72.0,57.0
13,681,567.0,NaN,100.0,14.0


In [89]:
shop_feaut.columns = ["total_count","click","add_car","buy","collect"]
shop_feaut.head()

,total_count,click,add_car,buy,collect
seller_id,,,,,
2,214,181.0,NaN,23.0,10.0
8,305,258.0,NaN,30.0,17.0
9,242,219.0,NaN,14.0,9.0
10,1658,1529.0,NaN,72.0,57.0
13,681,567.0,NaN,100.0,14.0


In [90]:
#商铺下交互的总用户数，多少被交互的商品，商品类别和商品品牌数量
shop_feaut_2 = user_log.groupby("seller_id")["user_id","item_id","cat_id","brand_id"].nunique()
shop_feaut_2.head()

,user_id,item_id,cat_id,brand_id
seller_id,,,,
2,82,48,7,1
8,83,51,8,1
9,72,103,13,15
10,410,185,13,2
13,187,28,5,1


In [91]:
shop_feaut_2.columns=["user_count","item_count","cat_count","brand_count"]
shop_feaut= shop_feaut.merge(shop_feaut_2,on="seller_id")
shop_feaut.head()

,total_count,click,add_car,buy,collect,user_count,item_count,cat_count,brand_count
seller_id,,,,,,,,,
2,214,181.0,NaN,23.0,10.0,82,48,7,1
8,305,258.0,NaN,30.0,17.0,83,51,8,1
9,242,219.0,NaN,14.0,9.0,72,103,13,15
10,1658,1529.0,NaN,72.0,57.0,410,185,13,2
13,681,567.0,NaN,100.0,14.0,187,28,5,1


In [92]:
#商铺月平均有多少用户交互
shop_feaut_2= user_log.groupby("seller_id")["month"].nunique()
shop_feaut= shop_feaut.merge(shop_feaut_2,on="seller_id")
shop_feaut.head()

,total_count,click,add_car,buy,collect,user_count,item_count,cat_count,brand_count,month
seller_id,,,,,,,,,,
2,214,181.0,NaN,23.0,10.0,82,48,7,1,7
8,305,258.0,NaN,30.0,17.0,83,51,8,1,7
9,242,219.0,NaN,14.0,9.0,72,103,13,15,7
10,1658,1529.0,NaN,72.0,57.0,410,185,13,2,7
13,681,567.0,NaN,100.0,14.0,187,28,5,1,7


In [93]:
shop_feaut["month_avg_user"]=shop_feaut.user_count/shop_feaut.month
shop_feaut.head()

,total_count,click,add_car,buy,collect,user_count,item_count,cat_count,brand_count,month,month_avg_user
seller_id,,,,,,,,,,,
2,214,181.0,NaN,23.0,10.0,82,48,7,1,7,11.714286
8,305,258.0,NaN,30.0,17.0,83,51,8,1,7,11.857143
9,242,219.0,NaN,14.0,9.0,72,103,13,15,7,10.285714
10,1658,1529.0,NaN,72.0,57.0,410,185,13,2,7,58.571429
13,681,567.0,NaN,100.0,14.0,187,28,5,1,7,26.714286


In [94]:
#获取用户详细信息
user_log = user_log.merge(user_info,on="user_id")
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,month,day,age_range,gender
0,26516,416965,1401,586,5579.0,1107,0,11,7,4.0,1.0
1,26516,416965,1401,586,5579.0,1107,0,11,7,4.0,1.0
2,26516,416965,1401,586,5579.0,1107,0,11,7,4.0,1.0
3,26516,352345,177,2565,8149.0,1101,0,11,1,4.0,1.0
4,26516,142231,177,2565,8149.0,1101,0,11,1,4.0,1.0


In [95]:
#商铺下交互的用户按年龄段和性别分别统计
shop_feaut_2 = pd.pivot_table(user_log,index="seller_id",columns="age_range",values="user_id",aggfunc="nunique")
shop_feaut_2.head()

age_range,1.0,2.0,3.0,4.0,5.0,6.0,7.0
seller_id,,,,,,,
2,NaN,10.0,19.0,14.0,9.0,8.0,2.0
8,NaN,6.0,23.0,17.0,8.0,7.0,2.0
9,NaN,8.0,21.0,19.0,6.0,11.0,2.0
10,NaN,17.0,108.0,80.0,55.0,40.0,10.0
13,NaN,25.0,51.0,19.0,19.0,21.0,7.0


In [96]:
shop_feaut_2.columns=["less18","between18and24","between25and29","between30and34","between35and39","between40and49","grate50"]
shop_feaut_2.head()

,less18,between18and24,between25and29,between30and34,between35and39,between40and49,grate50
seller_id,,,,,,,
2,NaN,10.0,19.0,14.0,9.0,8.0,2.0
8,NaN,6.0,23.0,17.0,8.0,7.0,2.0
9,NaN,8.0,21.0,19.0,6.0,11.0,2.0
10,NaN,17.0,108.0,80.0,55.0,40.0,10.0
13,NaN,25.0,51.0,19.0,19.0,21.0,7.0


In [97]:
shop_feaut= shop_feaut.merge(shop_feaut_2,on="seller_id")
shop_feaut.head()

,total_count,click,add_car,buy,collect,user_count,item_count,cat_count,brand_count,month,month_avg_user,less18,between18and24,between25and29,between30and34,between35and39,between40and49,grate50
seller_id,,,,,,,,,,,,,,,,,,
2,214,181.0,NaN,23.0,10.0,82,48,7,1,7,11.714286,NaN,10.0,19.0,14.0,9.0,8.0,2.0
8,305,258.0,NaN,30.0,17.0,83,51,8,1,7,11.857143,NaN,6.0,23.0,17.0,8.0,7.0,2.0
9,242,219.0,NaN,14.0,9.0,72,103,13,15,7,10.285714,NaN,8.0,21.0,19.0,6.0,11.0,2.0
10,1658,1529.0,NaN,72.0,57.0,410,185,13,2,7,58.571429,NaN,17.0,108.0,80.0,55.0,40.0,10.0
13,681,567.0,NaN,100.0,14.0,187,28,5,1,7,26.714286,NaN,25.0,51.0,19.0,19.0,21.0,7.0


In [98]:
shop_feaut_2 = pd.pivot_table(user_log,index="seller_id",columns="gender",values="user_id",aggfunc="nunique")
shop_feaut_2.head()

gender,0.0,1.0
seller_id,,
2,44.0,18.0
8,60.0,3.0
9,18.0,49.0
10,187.0,123.0
13,77.0,65.0


In [99]:
shop_feaut_2.columns=["gender_F","gender_M"]
shop_feaut= shop_feaut.merge(shop_feaut_2,on="seller_id")
shop_feaut.head()

,total_count,click,add_car,buy,collect,user_count,item_count,cat_count,brand_count,month,month_avg_user,less18,between18and24,between25and29,between30and34,between35and39,between40and49,grate50,gender_F,gender_M
seller_id,,,,,,,,,,,,,,,,,,,,
2,214,181.0,NaN,23.0,10.0,82,48,7,1,7,11.714286,NaN,10.0,19.0,14.0,9.0,8.0,2.0,44.0,18.0
8,305,258.0,NaN,30.0,17.0,83,51,8,1,7,11.857143,NaN,6.0,23.0,17.0,8.0,7.0,2.0,60.0,3.0
9,242,219.0,NaN,14.0,9.0,72,103,13,15,7,10.285714,NaN,8.0,21.0,19.0,6.0,11.0,2.0,18.0,49.0
10,1658,1529.0,NaN,72.0,57.0,410,185,13,2,7,58.571429,NaN,17.0,108.0,80.0,55.0,40.0,10.0,187.0,123.0
13,681,567.0,NaN,100.0,14.0,187,28,5,1,7,26.714286,NaN,25.0,51.0,19.0,19.0,21.0,7.0,77.0,65.0


In [100]:
#删除提取商铺特征的临时数据框
del shop_feaut_2

In [101]:
#合并提取的特征到电商--用户行为复购表
user_feaut = user_feaut.reset_index()
user_merchant = user_merchant.merge(user_feaut,on="user_id",how="left")
shop_feaut = shop_feaut.reset_index()
user_merchant = user_merchant.merge(shop_feaut,right_on="seller_id",left_on="merchant_id" ,how="left")

In [102]:
#删除用户，商铺特征提取表
del user_feaut
del shop_feaut

###  用户商铺交叉特征提取

In [103]:
user_shop_feaut = user_log.groupby(["seller_id","user_id"])["month"].count()
user_shop_feaut = user_shop_feaut.reset_index();
user_shop_feaut.columns= ["seller_id","user_id","us_total"]
user_shop_feaut .head()

,seller_id,user_id,us_total
0,2,1655,1
1,2,7232,4
2,2,10234,2
3,2,15325,7
4,2,15442,1


In [104]:
#用户在某商铺中各种行为次数（点击、加购、收藏和购买）
user_shop_feaut_2 = pd.pivot_table(user_log,index=["seller_id","user_id"],columns="action_type",values="day",aggfunc="count")
user_shop_feaut_2 = user_shop_feaut_2.reset_index()
user_shop_feaut_2.columns = ["seller_id","user_id","us_click","us_add_car","us_buy","us_collect"]
user_shop_feaut_2 .head()

,seller_id,user_id,us_click,us_add_car,us_buy,us_collect
0,2,1655,1.0,NaN,NaN,NaN
1,2,7232,3.0,NaN,1.0,NaN
2,2,10234,1.0,NaN,NaN,1.0
3,2,15325,6.0,NaN,1.0,NaN
4,2,15442,1.0,NaN,NaN,NaN


In [105]:
user_shop_feaut = user_shop_feaut.merge(user_shop_feaut_2,on=["seller_id","user_id"])
user_shop_feaut.head()

,seller_id,user_id,us_total,us_click,us_add_car,us_buy,us_collect
0,2,1655,1,1.0,NaN,NaN,NaN
1,2,7232,4,3.0,NaN,1.0,NaN
2,2,10234,2,1.0,NaN,NaN,1.0
3,2,15325,7,6.0,NaN,1.0,NaN
4,2,15442,1,1.0,NaN,NaN,NaN


In [106]:
#用户在商铺中月的平均交互次数 ,各种行为平均次数（点击、加购、收藏和购买）
user_shop_feaut_2 = user_log.groupby(["seller_id","user_id"])["month"].nunique()
user_shop_feaut_2 = user_shop_feaut_2.reset_index()
user_shop_feaut_2.columns =["seller_id","user_id","us_month"]
user_shop_feaut_2.head()

,seller_id,user_id,us_month
0,2,1655,1
1,2,7232,1
2,2,10234,1
3,2,15325,1
4,2,15442,1


In [107]:
user_shop_feaut = user_shop_feaut.merge(user_shop_feaut_2,on=["seller_id","user_id"])
user_shop_feaut.head()

,seller_id,user_id,us_total,us_click,us_add_car,us_buy,us_collect,us_month
0,2,1655,1,1.0,NaN,NaN,NaN,1
1,2,7232,4,3.0,NaN,1.0,NaN,1
2,2,10234,2,1.0,NaN,NaN,1.0,1
3,2,15325,7,6.0,NaN,1.0,NaN,1
4,2,15442,1,1.0,NaN,NaN,NaN,1


In [108]:
#用户在商铺中月的平均交互次数 ,各种行为平均次数（点击、加购、收藏和购买）
user_shop_feaut["us_month_avg_total"] =  user_shop_feaut.us_total/user_shop_feaut.us_month
user_shop_feaut["us_month_avg_click"] = user_shop_feaut.us_click/user_shop_feaut.us_month
user_shop_feaut["us_month_avg_addCar"] =np.where(user_shop_feaut.us_add_car.isnull(),0,user_shop_feaut.us_add_car/user_shop_feaut.us_month)
user_shop_feaut["us_month_avg_buy"] =np.where(user_shop_feaut.us_buy.isnull(),0,user_shop_feaut.us_buy/user_shop_feaut.us_month)
user_shop_feaut["us_month_avg_collect"] =np.where(user_shop_feaut.us_collect.isnull(),0,user_shop_feaut.us_collect/user_shop_feaut.us_month)
user_shop_feaut.head()

,seller_id,user_id,us_total,us_click,us_add_car,us_buy,us_collect,us_month,us_month_avg_total,us_month_avg_click,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect
0,2,1655,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0
1,2,7232,4,3.0,NaN,1.0,NaN,1,4.0,3.0,0.0,1.0,0.0
2,2,10234,2,1.0,NaN,NaN,1.0,1,2.0,1.0,0.0,0.0,1.0
3,2,15325,7,6.0,NaN,1.0,NaN,1,7.0,6.0,0.0,1.0,0.0
4,2,15442,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0


In [109]:
user_shop_feaut_2 = user_log.groupby(["seller_id","user_id"])["time_stamp"].agg(["min","max"])
user_shop_feaut_2 = user_shop_feaut_2.reset_index()
user_shop_feaut_2.columns=["seller_id","user_id","us_min","us_max"]
user_shop_feaut_2.head()

,seller_id,user_id,us_min,us_max
0,2,1655,1111,1111
1,2,7232,1107,1107
2,2,10234,911,912
3,2,15325,1104,1111
4,2,15442,1111,1111


In [110]:
user_shop_feaut_2["diff"] = user_shop_feaut_2.us_max-user_shop_feaut_2.us_min
user_shop_feaut_2.head()

,seller_id,user_id,us_min,us_max,diff
0,2,1655,1111,1111,0
1,2,7232,1107,1107,0
2,2,10234,911,912,1
3,2,15325,1104,1111,7
4,2,15442,1111,1111,0


In [111]:
user_shop_feaut_2 = user_shop_feaut_2.loc[:,["seller_id","user_id","diff"]]
user_shop_feaut = user_shop_feaut.merge(user_shop_feaut_2,on=["seller_id","user_id"])
user_shop_feaut.head()

,seller_id,user_id,us_total,us_click,us_add_car,us_buy,us_collect,us_month,us_month_avg_total,us_month_avg_click,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff
0,2,1655,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0
1,2,7232,4,3.0,NaN,1.0,NaN,1,4.0,3.0,0.0,1.0,0.0,0
2,2,10234,2,1.0,NaN,NaN,1.0,1,2.0,1.0,0.0,0.0,1.0,1
3,2,15325,7,6.0,NaN,1.0,NaN,1,7.0,6.0,0.0,1.0,0.0,7
4,2,15442,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0


In [112]:
#用户在商铺中交互有多少天
user_shop_feaut_2 = user_log.groupby(["seller_id","user_id"])["time_stamp"].nunique()
user_shop_feaut_2 = user_shop_feaut_2.reset_index();
user_shop_feaut_2.columns = ["seller_id","user_id","us_days"]
user_shop_feaut = user_shop_feaut.merge(user_shop_feaut_2,on=["seller_id","user_id"])
user_shop_feaut.head()

,seller_id,user_id,us_total,us_click,us_add_car,us_buy,us_collect,us_month,us_month_avg_total,us_month_avg_click,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff,us_days
0,2,1655,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0,1
1,2,7232,4,3.0,NaN,1.0,NaN,1,4.0,3.0,0.0,1.0,0.0,0,1
2,2,10234,2,1.0,NaN,NaN,1.0,1,2.0,1.0,0.0,0.0,1.0,1,2
3,2,15325,7,6.0,NaN,1.0,NaN,1,7.0,6.0,0.0,1.0,0.0,7,3
4,2,15442,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0,1


In [113]:
#用户在商铺中交互的商品、商品类别和商品品牌的总个数
user_shop_feaut_2 = user_log.groupby(["seller_id","user_id"])["item_id","cat_id",'brand_id'].nunique()
user_shop_feaut_2 = user_shop_feaut_2.reset_index()
user_shop_feaut_2.columns=["seller_id","user_id","us_item_count","us_cat_count","us_brand_count"]
user_shop_feaut = user_shop_feaut.merge(user_shop_feaut_2,on=["seller_id","user_id"])
user_shop_feaut.head()

,seller_id,user_id,us_total,us_click,us_add_car,us_buy,us_collect,us_month,us_month_avg_total,us_month_avg_click,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff,us_days,us_item_count,us_cat_count,us_brand_count
0,2,1655,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0,1,1,1,1
1,2,7232,4,3.0,NaN,1.0,NaN,1,4.0,3.0,0.0,1.0,0.0,0,1,3,1,1
2,2,10234,2,1.0,NaN,NaN,1.0,1,2.0,1.0,0.0,0.0,1.0,1,2,1,1,1
3,2,15325,7,6.0,NaN,1.0,NaN,1,7.0,6.0,0.0,1.0,0.0,7,3,1,1,1
4,2,15442,1,1.0,NaN,NaN,NaN,1,1.0,1.0,0.0,0.0,0.0,0,1,1,1,1


In [114]:
del user_shop_feaut_2

In [115]:
#合并用户-商铺特征（user_shop_feaut） 数据框到user_merchant 表内
user_merchant = user_merchant.merge(user_shop_feaut,right_on=["seller_id","user_id"],left_on=["merchant_id","user_id"])
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_month_avg_total,us_month_avg_click,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff,us_days,us_item_count,us_cat_count,us_brand_count
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,1.0,NaN,0.0,1.0,0.0,0,1,1,1,1
1,314568,3828,0,308,284.0,NaN,12.0,12.0,23,123,...,18.0,16.0,0.0,1.0,1.0,1,2,7,5,1
2,314580,3853,0,21,19.0,NaN,2.0,NaN,9,10,...,10.0,9.0,0.0,1.0,0.0,2,3,1,1,1
3,339322,1399,0,10,9.0,NaN,1.0,NaN,7,7,...,3.0,2.0,0.0,1.0,0.0,0,1,1,1,2
4,314555,3828,0,54,45.0,NaN,2.0,7.0,14,23,...,4.5,3.5,0.0,0.5,0.5,80,6,2,2,1


In [116]:
#删除掉 用户_商铺特征表
del user_shop_feaut

### 比值特征提取

In [117]:
#用户交互次数在所有用户交互总次数的占比
all_user_log = user_log.shape[0]
user_merchant["user_total_log_p"] = user_merchant.total_log/all_user_log
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_month_avg_click,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff,us_days,us_item_count,us_cat_count,us_brand_count,user_total_log_p
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,NaN,0.0,1.0,0.0,0,1,1,1,1,0.000008
1,314568,3828,0,308,284.0,NaN,12.0,12.0,23,123,...,16.0,0.0,1.0,1.0,1,2,7,5,1,0.000201
2,314580,3853,0,21,19.0,NaN,2.0,NaN,9,10,...,9.0,0.0,1.0,0.0,2,3,1,1,1,0.000014
3,339322,1399,0,10,9.0,NaN,1.0,NaN,7,7,...,2.0,0.0,1.0,0.0,0,1,1,1,2,0.000007
4,314555,3828,0,54,45.0,NaN,2.0,7.0,14,23,...,3.5,0.0,0.5,0.5,80,6,2,2,1,0.000035


In [118]:
#用户购买次数在所有用户购买总次数的占比
all_user_log_buy = (user_log.action_type==2).sum()
user_merchant["user_total_buy_p"] = user_merchant.buy_x/all_user_log_buy
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff,us_days,us_item_count,us_cat_count,us_brand_count,user_total_log_p,user_total_buy_p
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,0.0,1.0,0.0,0,1,1,1,1,0.000008,0.000049
1,314568,3828,0,308,284.0,NaN,12.0,12.0,23,123,...,0.0,1.0,1.0,1,2,7,5,1,0.000201,0.000118
2,314580,3853,0,21,19.0,NaN,2.0,NaN,9,10,...,0.0,1.0,0.0,2,3,1,1,1,0.000014,0.000020
3,339322,1399,0,10,9.0,NaN,1.0,NaN,7,7,...,0.0,1.0,0.0,0,1,1,1,2,0.000007,0.000010
4,314555,3828,0,54,45.0,NaN,2.0,7.0,14,23,...,0.0,0.5,0.5,80,6,2,2,1,0.000035,0.000020


In [119]:
#用户交互次数在所有用户交互总次数的占比
all_user_log = user_log.shape[0]
user_merchant["user_total_log_p"] = user_merchant.total_log/all_user_log
#用户购买次数在所有用户购买总次数的占比
all_user_log_buy = (user_log.action_type==2).sum()
user_merchant["user_total_buy_p"] = user_merchant.buy_x/all_user_log_buy
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_month_avg_addCar,us_month_avg_buy,us_month_avg_collect,diff,us_days,us_item_count,us_cat_count,us_brand_count,user_total_log_p,user_total_buy_p
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,0.0,1.0,0.0,0,1,1,1,1,0.000008,0.000049
1,314568,3828,0,308,284.0,NaN,12.0,12.0,23,123,...,0.0,1.0,1.0,1,2,7,5,1,0.000201,0.000118
2,314580,3853,0,21,19.0,NaN,2.0,NaN,9,10,...,0.0,1.0,0.0,2,3,1,1,1,0.000014,0.000020
3,339322,1399,0,10,9.0,NaN,1.0,NaN,7,7,...,0.0,1.0,0.0,0,1,1,1,2,0.000007,0.000010
4,314555,3828,0,54,45.0,NaN,2.0,7.0,14,23,...,0.0,0.5,0.5,80,6,2,2,1,0.000035,0.000020


In [120]:
#商铺中交互次数在所有商铺交互次数的占比
user_merchant["shop_total_log_p"] = user_merchant.total_count/all_user_log
#商铺中交购买次数在所有商铺购买总次数的占比
user_merchant["shop_total_buy_p"] = user_merchant.buy_y/all_user_log_buy
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_month_avg_collect,diff,us_days,us_item_count,us_cat_count,us_brand_count,user_total_log_p,user_total_buy_p,shop_total_log_p,shop_total_buy_p
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,0.0,0,1,1,1,1,0.000008,0.000049,0.002593,0.003010
1,314568,3828,0,308,284.0,NaN,12.0,12.0,23,123,...,1.0,1,2,7,5,1,0.000201,0.000118,0.050034,0.022763
2,314580,3853,0,21,19.0,NaN,2.0,NaN,9,10,...,0.0,2,3,1,1,1,0.000014,0.000020,0.001956,0.000984
3,339322,1399,0,10,9.0,NaN,1.0,NaN,7,7,...,0.0,0,1,1,1,2,0.000007,0.000010,0.000787,0.000708
4,314555,3828,0,54,45.0,NaN,2.0,7.0,14,23,...,0.5,80,6,2,2,1,0.000035,0.000020,0.050034,0.022763


In [121]:
shop_featu =  user_log.loc[user_log.action_type==2].groupby("seller_id")["user_id"].nunique()
shop_featu = shop_featu.reset_index()
shop_featu.head()

,seller_id,user_id
0,2,10
1,8,11
2,9,8
3,10,44
4,13,31


In [122]:
shop_featu.columns = ["seller_id","Shop_buy_users"]
shop_featu.head()

,seller_id,Shop_buy_users
0,2,10
1,8,11
2,9,8
3,10,44
4,13,31


In [123]:
user_merchant = user_merchant.merge(shop_featu,left_on="seller_id_y",right_on="seller_id")
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_days,us_item_count,us_cat_count,us_brand_count,user_total_log_p,user_total_buy_p,shop_total_log_p,shop_total_buy_p,seller_id,Shop_buy_users
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,1,1,1,1,0.000008,0.000049,0.002593,0.00301,3609,126
1,395590,3609,0,51,42.0,NaN,5.0,4.0,10,24,...,3,12,1,1,0.000033,0.000049,0.002593,0.00301,3609,126
2,395570,3609,0,23,19.0,NaN,3.0,1.0,6,10,...,1,1,1,1,0.000015,0.000030,0.002593,0.00301,3609,126
3,250943,3609,0,36,30.0,NaN,6.0,NaN,14,24,...,1,1,1,1,0.000023,0.000059,0.002593,0.00301,3609,126
4,126392,3609,0,85,81.0,NaN,4.0,NaN,17,28,...,1,1,1,1,0.000055,0.000039,0.002593,0.00301,3609,126


In [124]:
#商铺中购买的用户数在所有商铺购买用户数的占比
all_buy_user_count =user_log.user_id[user_log.action_type==2].nunique()
user_merchant["shop_buyuser_totalbuyuser_p"] = user_merchant.Shop_buy_users/all_buy_user_count
user_merchant.head()


,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,us_item_count,us_cat_count,us_brand_count,user_total_log_p,user_total_buy_p,shop_total_log_p,shop_total_buy_p,seller_id,Shop_buy_users,shop_buyuser_totalbuyuser_p
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,1,1,1,0.000008,0.000049,0.002593,0.00301,3609,126,0.005451
1,395590,3609,0,51,42.0,NaN,5.0,4.0,10,24,...,12,1,1,0.000033,0.000049,0.002593,0.00301,3609,126,0.005451
2,395570,3609,0,23,19.0,NaN,3.0,1.0,6,10,...,1,1,1,0.000015,0.000030,0.002593,0.00301,3609,126,0.005451
3,250943,3609,0,36,30.0,NaN,6.0,NaN,14,24,...,1,1,1,0.000023,0.000059,0.002593,0.00301,3609,126,0.005451
4,126392,3609,0,85,81.0,NaN,4.0,NaN,17,28,...,1,1,1,0.000055,0.000039,0.002593,0.00301,3609,126,0.005451


In [125]:
#用户在某商铺的交互次数在该用户所有交互次数的占比
user_merchant["us_user_log_p"] = user_merchant.us_total/user_merchant.total_log
#用户在某商铺的购买次数在该用户所有购买次数的占比
user_merchant["us_user_buy_p"] = user_merchant.us_buy/user_merchant.buy_x
#用户在某商铺的交互次数在该商铺所有交互次数中的占比
user_merchant["us_shop_log_p"] = user_merchant.us_total/user_merchant.total_count
#用户在某商铺的购买次数在该商铺所有购买次数中的占比
user_merchant["us_shop_buy_p"] = user_merchant.us_buy/user_merchant.buy_y
user_merchant.head()

,user_id,merchant_id,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,...,user_total_buy_p,shop_total_log_p,shop_total_buy_p,seller_id,Shop_buy_users,shop_buyuser_totalbuyuser_p,us_user_log_p,us_user_buy_p,us_shop_log_p,us_shop_buy_p
0,251021,3609,0,12,7.0,NaN,5.0,NaN,5,6,...,0.000049,0.002593,0.00301,3609,126,0.005451,0.083333,0.200000,0.000252,0.003268
1,395590,3609,0,51,42.0,NaN,5.0,4.0,10,24,...,0.000049,0.002593,0.00301,3609,126,0.005451,0.490196,0.400000,0.006292,0.006536
2,395570,3609,0,23,19.0,NaN,3.0,1.0,6,10,...,0.000030,0.002593,0.00301,3609,126,0.005451,0.130435,0.333333,0.000755,0.003268
3,250943,3609,0,36,30.0,NaN,6.0,NaN,14,24,...,0.000059,0.002593,0.00301,3609,126,0.005451,0.055556,0.166667,0.000503,0.003268
4,126392,3609,0,85,81.0,NaN,4.0,NaN,17,28,...,0.000039,0.002593,0.00301,3609,126,0.005451,0.129412,0.250000,0.002769,0.003268


In [126]:
user_log["uid_day"] = user_log.user_id.astype("str") +"_"+ user_log.time_stamp.astype("str")
user_log.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,month,day,age_range,gender,uid_day
0,26516,416965,1401,586,5579.0,1107,0,11,7,4.0,1.0,26516_1107
1,26516,416965,1401,586,5579.0,1107,0,11,7,4.0,1.0,26516_1107
2,26516,416965,1401,586,5579.0,1107,0,11,7,4.0,1.0,26516_1107
3,26516,352345,177,2565,8149.0,1101,0,11,1,4.0,1.0,26516_1101
4,26516,142231,177,2565,8149.0,1101,0,11,1,4.0,1.0,26516_1101


In [127]:
#商铺中回购（购买次数>1）的用户数占总回购用户数的比例
#认为一天的为一个订单
#总回购用户数
temp = user_log.uid_day[user_log.action_type==2].unique()
buy_double_user = (pd.Series(map(lambda x:x.split("_")[0],temp)).value_counts()>1).sum()

In [188]:
#定义函数获得指定店铺的回归用户数
def get_shop_double(shop_id):
    temp = user_log.uid_day[(user_log.action_type==2) & (user_log.seller_id == shop_id)].unique()
    # print(temp)
    # print(pd.Series(map(lambda x:x.split("_")[0],temp)).value_counts())
    buy_double_user = (pd.Series(map(lambda x:x.split("_")[0],temp)).value_counts()>1).sum()
    return [shop_id,buy_double_user]

In [213]:
get_shop_double(586)

[586, 9]

In [190]:
#遍历每个店铺获得其回购用户数
temp = pd.DataFrame(map(get_shop_double,user_log.seller_id.unique()),columns=["seller_id","double_user"])
temp.head()

,seller_id,double_user
0,586,9
1,2565,6
2,1892,15
3,862,0
4,2403,1


In [192]:
user_merchant = user_merchant.merge(temp,right_on="seller_id",left_on="merchant_id")
user_merchant.head()

KeyError: 'merchant_id'

In [193]:
#计算比值
user_merchant["shop_doubleU_totaldoubleU_p"] = user_merchant.double_user/buy_double_user
user_merchant.head()

,label,total_log,click_x,add_car_x,buy_x,collect_x,seller_count,item_count_x,cat_count_x,brand_count_x,...,us_user_log_p,us_user_buy_p,us_shop_log_p,us_shop_buy_p,double_user,shop_doubleU_totaldoubleU_p,shop_Repurchase_P,shop_Repurchase_cat_P,User_RepurchaseShop_P,User_Repurchase_P
0,0,12,7.0,0.0,5.0,0.0,5,6,5,5,...,0.083333,0.200000,0.000252,0.003268,8,0.000565,0.1,0.986111,0.000000,0.666667
1,1,12,7.0,0.0,5.0,0.0,5,6,5,5,...,0.083333,0.200000,0.003436,0.027778,0,0.000000,0.0,0.884615,0.000000,0.666667
2,0,51,42.0,0.0,5.0,4.0,10,24,12,10,...,0.490196,0.400000,0.006292,0.006536,8,0.000565,0.1,0.986111,0.666667,0.666667
3,0,23,19.0,0.0,3.0,1.0,6,10,5,6,...,0.130435,0.333333,0.000755,0.003268,8,0.000565,0.1,0.986111,0.500000,0.500000
4,0,36,30.0,0.0,6.0,0.0,14,24,15,15,...,0.055556,0.166667,0.000503,0.003268,8,0.000565,0.1,0.986111,0.200000,0.500000


In [ ]:
#商铺中回购的总次数占商铺中用户总购买次数的比例
#获得指定店铺的回购用户比
def get_shop_double_order(shop_id):
    temp = user_log.uid_day[(user_log.action_type==2) & (user_log.seller_id == shop_id)].unique()
    temp = pd.Series(map(lambda x:x.split("_")[0],temp)).value_counts()
    total_buy = temp.sum()
    double = (temp-1).sum()
    return [shop_id,double/total_buy]

In [ ]:
get_shop_double_order(586)

In [200]:
#遍历每一个店铺，获得其商铺中回购的总次数占商铺中用户总购买次数的比例
temp = pd.DataFrame(map(get_shop_double_order,user_log.seller_id.unique()),columns=["merchant_id","shop_Repurchase_P"])
temp.head()

,merchant_id,shop_Repurchase_P
0,586,0.078125
1,2565,0.047059
2,1892,0.060052
3,862,0.000000
4,2403,0.013158


,merchant_id,shop_Repurchase_P
0,586,0.078125
1,2565,0.047059
2,1892,0.060052
3,862,0.000000
4,2403,0.013158


,merchant_id,shop_Repurchase_P
0,586,0.078125
1,2565,0.047059
2,1892,0.060052
3,862,0.000000
4,2403,0.013158


,merchant_id,shop_Repurchase_P
0,586,0.078125
1,2565,0.047059
2,1892,0.060052
3,862,0.000000
4,2403,0.013158


,merchant_id,shop_Repurchase_P
0,586,0.078125
1,2565,0.047059
2,1892,0.060052
3,862,0.000000
4,2403,0.013158


In [210]:
user_merchant = user_merchant.merge(temp,on="merchant_id")
user_merchant.head()

KeyError: 'merchant_id'

In [224]:
#商铺中回购的商品类别总数在商铺中用户购买商品类别总数的占比
def get_shop_double_cat(shop_id):
    temp = user_log.cat_id[(user_log.action_type==2) & (user_log.seller_id == shop_id)].value_counts()
    print(temp-1)
    zong = temp.sum()
    fugou = (temp-1).sum()
    return [shop_id,fugou/zong]

In [ ]:
#遍历所有商铺，获得其品类回购率
temp =pd.DataFrame(list(map(get_shop_double_cat,user_log.seller_id.unique())),columns=["merchant_id","shop_Repurchase_cat_P"])
user_merchant = user_merchant.merge(temp,on="merchant_id")
user_merchant.head()

656     105
1467     91
1271     35
163       1
Name: cat_id, dtype: int64
177    41
11      4
Name: cat_id, dtype: int64
898     107
1528     20
1644     16
278      11
681      10
1544      7
22        3
96        2
730       2
484       2
1338      1
698       1
955       1
1083      1
1650      1
331       0
1058      0
1328      0
1370      0
1626      0
430       0
532       0
1187      0
386       0
517       0
499       0
1414      0
1100      0
753       0
1540      0
1357      0
1292      0
1192      0
Name: cat_id, dtype: int64
177    21
Name: cat_id, dtype: int64
812     24
1028    17
420      6
629      3
1036     1
351      1
1260     0
1174     0
1658     0
Name: cat_id, dtype: int64
898     57
1644    17
1500     9
1528     7
944      6
43       6
513      4
151      2
1401     2
786      1
753      1
1422     1
1362     1
304      1
1517     1
77       0
331      0
1085     0
493      0
192      0
115      0
Name: cat_id, dtype: int64
1252    15
1159     7
1228     5
1

KeyError: 'merchant_id'

In [226]:
#用户有回购的商铺数在该用户所有购买行为商铺数的占比
def get_shop_double_us(user_id):
    temp = user_log.seller_id[(user_log.action_type==2) & (user_log.user_id == user_id)].value_counts()
    zong = temp.size
    fugou = (temp>1).sum()
    return [user_id,fugou/zong]

In [268]:
#遍历所有用户 ，获得其有回购的商铺数在该用户所有购买行为商铺数的占比
temp =pd.DataFrame(list(map(get_shop_double_us,user_log.user_id.unique())),columns=["user_id","User_RepurchaseShop_P"])
user_merchant = user_merchant.merge(temp,on="user_id")
user_merchant.head()

KeyboardInterrupt: 

In [ ]:
#用户回购次数在该用户所有购买总次数的占比
def get_User_Repurchase(user_id):
    temp = user_log.uid_day[(user_log.action_type==2) & (user_log.user_id == user_id)].unique()
    temp = pd.Series(map(lambda x:x.split("_")[0],temp)).value_counts()
    total_buy = temp.sum()
    double = (temp-1).sum()
    return [user_id,double/total_buy]

In [ ]:
get_User_Repurchase(251021)

In [ ]:
user_log.loc[(user_log.action_type==2) & (user_log.user_id == 251021),:]

In [278]:
#遍历所有用户 ，获得其回购次数在该用户所有购买总次数的占比
temp =pd.DataFrame(list(map(get_User_Repurchase,user_log.user_id.unique())),columns=["user_id","User_Repurchase_P"])
user_merchant = user_merchant.merge(temp,on="user_id")
user_merchant.head()

AttributeError: 'DataFrame' object has no attribute 'uid_day'

In [ ]:
#缺失值设置为0
user_merchant.fillna(0,inplace=True)
#删除不需要的列
user_merchant = user_merchant.drop(["seller_id_x","seller_id_y","user_id","merchant_id"],axis=1)
user_merchant.columns

In [277]:
user_merchant


,user_id,merchant_id,label
0,251021,3609,0
1,251038,3624,0
2,314568,3828,0
3,314580,3853,0
4,339322,1399,0
...,...,...,...
31899,265534,102,1
31900,178906,102,1
31901,178083,141,1
31902,337700,722,1


# 建模及评估

In [2]:
import os
import time
# 模型处理模块
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
# 标准化处理模块
from sklearn.preprocessing import StandardScaler
# 常规模型
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# 集成学习和stacking模型
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from mlxtend.classifier import StackingClassifier
# 评价标准模块
from sklearn import metrics
from sklearn.metrics import accuracy_score,roc_auc_score,recall_score,precision_score
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/Users/abc/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [149]:
X_train,X_test,y_train,y_test = train_test_split(user_merchant.iloc[:,1:],user_merchant.label,test_size=0.3,random_state=6)

In [5]:
def train_model(X_train, y_train, X_test, y_test,
               model,model_name):

    print('训练{}'.format(model_name))

    clf=model
    start = time.time()
    clf.fit(X_train, y_train.values.ravel())

     #验证模型
    print('训练准确率：{:.4f}'.format(clf.score(X_train, y_train)))


    predict=clf.predict(X_test)
    score = clf.score(X_test, y_test)
    precision=precision_score(y_test,predict)
    recall=recall_score(y_test,predict)
    print('测试准确率：{:.4f}'.format(score))
    print('测试精确率：{:.4f}'.format(precision))
    print('测试召回率：{:.4f}'.format(recall))

    end = time.time()
    duration = end - start
    print('模型训练耗时：{:6f}s'.format(duration))


    return clf, score,precision,recall, duration

In [6]:
model_name_param_dict = {    'LR': (LogisticRegression(penalty ="l2")),
                             'DT': (DecisionTreeClassifier(max_depth=10,min_samples_split=10)),
                             'AdaBoost': (AdaBoostClassifier()),
                             'GBDT': (GradientBoostingClassifier()),
                             'RF': (RandomForestClassifier()),
                             'XGBoost':(XGBClassifier())
                         }

result_df = pd.DataFrame(columns=['Accuracy (%)','precision(%)','recall(%)','Time (s)'],
                             index=list(model_name_param_dict.keys()))

for model_name, model in model_name_param_dict.items():
    clf, acc,pre,recall, mean_duration = train_model(X_train, y_train,
                                                        X_test, y_test,
                                                        model,model_name)
    result_df.loc[model_name, 'Accuracy (%)'] = acc
    result_df.loc[model_name, 'precision(%)'] = pre
    result_df.loc[model_name, 'recall(%)'] = recall
    result_df.loc[model_name, 'Time (s)'] = mean_duration

result_df.to_csv(os.path.join('model_comparison.csv'))

NameError: name 'X_train' is not defined

In [7]:
X_train.shape

NameError: name 'X_train' is not defined

In [83]:
#随机森林
#多个决策树组成，尽量使多个树之间是有差异的
#使多个树之间是有差异 1.样本抽样， 2. 特征数
#控制树过大（怕产生过拟合）---预剪枝
#1. 最大深度（树的层数，不包含叶子层）
#2. 分裂所需的最小样本数
#3. 叶节点最小样本数
#n_estimators  子模型的数量(树的个数)
#max_features  节点分裂时参与判断的最大特征数
#max_depth  最大深度
#min_samples_split 分裂所需的最小样本数
#min_samples_leaf  叶节点最小样本数
#bootstrap 是否bootstrap对样本抽样  False：子模型的样本一致，子模型间强相关  True：默认值
param_grid = {'n_estimators': [20, 50, 100,300], 'max_features': [10,20,30,40,50,60],"max_depth":[4,6,8,10,12],
             "min_samples_split": [10,20,30,40],"min_samples_leaf": [5,10,20,30]},
#为模型能正常创建，可以少设置几个参数选项，让其跑通代码
param_grid = {'n_estimators': [20], 'max_features': [60],"max_depth":[6,8],
             "min_samples_split": [40],"min_samples_leaf": [30]},
#4 * 6 * 5 * 4 * 4 * 5
model = RandomForestClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
result = grid_search.fit(X_train, y_train)
result.best_params_

NameError: name 'RandomForestClassifier' is not defined

In [154]:
result.best_score_

0.65698744551347

In [155]:
pre = result.predict(X_test)

In [156]:
#精准率和召回率    在提供的有限参数里，模型质量没有默认的好
print(precision_score(y_test,pre),recall_score(y_test,pre))

0.6073495370370371 0.5795140806184429


In [3]:
#XGBoost(提升树) 弱学习器： cart树（分类回归树）
#串行的（第二棵树是在第一棵树的基础上训练，训练的是第一棵树差值）
#XGBoost
#1. 损失函数
#2. 一阶导数g, 二阶导数h (确定叶子节点权重，分支节点的特征)
#3. 输出就是 y1 + y2 + y3
#调参
#1 剪枝： 最大深度，分裂所需的最小样本数，叶节点最小样本数
#2 损失函数
#3 棵树
#4 数据输入： 记录数据占全部训练集的比例，特征占全部特征的比例
#n_estimatores 即决策树的个数
#max_depth 树的深度，默认值为6，典型值3-10。
#subsample 训练每棵树时，使用的数据占全部训练集的比例。默认值为1，典型值为0.5-1。
#colsample_bytree 训练每棵树时，使用的特征占全部特征的比例。默认值为1，典型值为0.5-1。
#objective 选定损失函数
param_grid = {'n_estimators': [20, 50, 100,300],"max_depth":[4,6,8,10,12],
             "subsample": [0.3,0.5,0.6,0.7,0.8],"colsample_bytree": [0.3,0.5,0.6,0.7,0.8]},
model = XGBClassifier()
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='roc_auc')
temp=grid_search.fit(X_train, y_train)
temp.best_params_

NameError: name 'X_train' is not defined

In [4]:
pre = temp.predict(X_test)
#精准率和召回率    在提供的有限参数里，模型质量没有默认的好
print(precision_score(y_test,pre),recall_score(y_test,pre))

NameError: name 'temp' is not defined

In [270]:
#优质模型保存
from sklearn.externals import joblib
#保存模型
joblib.dump(temp,'model.model')

#加载模型
#clf=joblib.load('model.model')

ImportError: cannot import name 'joblib' from 'sklearn.externals' (/Users/abc/opt/anaconda3/lib/python3.8/site-packages/sklearn/externals/__init__.py)